<a href="https://colab.research.google.com/github/bhanup6663/COMP691_DL/blob/bhanu/XGBoost__Bayesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import Grayscale
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import torch

In [2]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    # Grayscale(num_output_channels=1),  # Convert images to grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.2])  # Adjust normalization for grayscale images
])

test_transform = transforms.Compose([
    # Grayscale(num_output_channels=1),  # Convert images to grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.2])  # Adjust normalization
])

In [3]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
selected_classes = np.random.choice(range(10), 2, replace=False)

In [5]:
class_mapping = {original: new for new, original in enumerate(selected_classes)}

In [6]:
train_indices = []
test_indices = []

for idx, (image, label) in enumerate(trainset):
    if label in selected_classes:  # Include all samples from selected classes for training
        train_indices.append(idx)

for idx, (image, label) in enumerate(testset):
    if label in selected_classes:  # Include all samples from selected classes for testing
        test_indices.append(idx)

# Shuffle the indices
np.random.shuffle(train_indices)
np.random.shuffle(test_indices)

# Check if enough samples are found for training and testing
if len(train_indices) < 25 or len(test_indices) < 2000:
    raise ValueError("Insufficient samples found for training or testing.")

In [7]:
train_indices = train_indices[:25]
test_indices = test_indices[:2000]

trainloader = torch.utils.data.DataLoader(trainset, batch_size=25, sampler=torch.utils.data.SubsetRandomSampler(train_indices))
testloader = torch.utils.data.DataLoader(testset, batch_size=2000, sampler=torch.utils.data.SubsetRandomSampler(test_indices))

In [8]:
train_data = []
train_labels = []
test_data = []
test_labels = []

for images, labels in trainloader:
    for image, label in zip(images, labels):
        train_data.append(image.numpy().flatten())  # Flatten the image tensor
        train_labels.append(label.item())

for images, labels in testloader:
    for image, label in zip(images, labels):
        test_data.append(image.numpy().flatten())  # Flatten the image tensor
        test_labels.append(label.item())

In [9]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

train_remapped = np.array([class_mapping[label] for label in train_labels])
test_remapped = np.array([class_mapping[label] for label in test_labels])

In [10]:
pca = PCA(n_components=0.70)
X_train_pca = pca.fit_transform(np.array(train_data))
X_test_pca = pca.transform(np.array(test_data))

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_train_pca, np.array(train_remapped), test_size=0.2, stratify=train_remapped)


In [12]:
def objective(space):
    model = XGBClassifier(
        n_estimators=int(space['n_estimators']),
        max_depth=int(space['max_depth']),
        learning_rate=space['learning_rate'],
        subsample=space['subsample'],
        colsample_bytree=space['colsample_bytree'],
        reg_lambda=space['lambda'],
        objective='multi:softmax',
        eval_metric='mlogloss',
        use_label_encoder=False,
        num_class=2,  # Make sure this is correctly set
        tree_method='gpu_hist',
        device='gpu',  # Updated from gpu_id to device
        seed=42,
        early_stopping_rounds=10  # Moved here as per deprecation warning
    )

    eval_set = [(X_val, y_val)]
    model.fit(X_train, y_train, eval_set=eval_set, verbose=False)

    predictions = model.predict(X_val)
    accuracy = accuracy_score(y_val, predictions)
    return {'loss': -accuracy, 'status': STATUS_OK}

In [13]:
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 300, 25),  # Widen the range to allow more trees
    'max_depth': hp.choice('max_depth', [3, 4, 5, 6, 7]),  # Narrow down based on typical good performers
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),  # Focus on a range that's typically more effective
    'subsample': hp.uniform('subsample', 0.5, 1.0),  # Narrow to focus on higher subsampling rates
    'colsample_bytree': hp.uniform('colsample_bytree', 0.4, 0.8),  # Adjust to focus on a moderate range
    'lambda': hp.uniform('lambda', 0.5, 2),  # Broaden to explore the impact of regularization more
}


In [14]:
trials = Trials()
best_hyperparams = fmin(fn=objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=100,
                        trials=trials)

print(f"Best hyperparameters: {best_hyperparams}")

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:14:08] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)



  1%|          | 1/100 [00:00<00:35,  2.81trial/s, best loss: -0.6]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:14:08] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:14:08] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:14:08] WARNING: /workspace/s

  4%|▍         | 4/100 [00:00<00:16,  5.83trial/s, best loss: -1.0]

...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:14:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:14:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:14:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, Use

 98%|█████████▊| 98/100 [00:13<00:00,  7.63trial/s, best loss: -1.0]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:14:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)



100%|██████████| 100/100 [00:13<00:00,  7.18trial/s, best loss: -1.0]
Best hyperparameters: {'colsample_bytree': 0.6720041970103184, 'lambda': 1.7722945120166442, 'learning_rate': 0.08508700068442625, 'max_depth': 2, 'n_estimators': 125.0, 'subsample': 0.6026313743310269}


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:14:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)



In [15]:
best_n_estimators = int(best_hyperparams['n_estimators'])
best_max_depth = int(best_hyperparams['max_depth'])
best_learning_rate = best_hyperparams['learning_rate']
best_subsample = best_hyperparams['subsample']
best_colsample_bytree = best_hyperparams['colsample_bytree']
best_lambda = best_hyperparams['lambda']

In [16]:
trained_model = XGBClassifier(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    learning_rate=best_learning_rate,
    subsample=best_subsample,
    colsample_bytree=best_colsample_bytree,
    reg_lambda=best_lambda,
    objective='multi:softmax',
    eval_metric='mlogloss',
    use_label_encoder=False,
    num_class=2,  # Assuming binary or two-class classification
    tree_method='gpu_hist',
    device='gpu',  # Or 'cpu' if not using GPU
    seed=42
)


In [17]:
trained_model.fit(X_train_pca, train_remapped)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:14:22] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6720041970103184, device='gpu',
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08508700068442625,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=125, n_jobs=None, num_class=2,
              num_parallel_tree=None, ...)

In [18]:
predictions = trained_model.predict(X_test_pca)

In [19]:
accuracy = accuracy_score(test_remapped, predictions)
print(f'XGBoost Test Accuracy: {accuracy*100:.2f}%')

XGBoost Test Accuracy: 78.60%
